In [1]:
# Povm

In [1]:
import pickle
import time
from typing import List
import datetime as dt
from pathlib import Path

import numpy as np
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px

from tqdm import tqdm

import numpy.testing as npt

from quara.data_analysis import data_analysis, physicality_violation_check, report
from quara.objects.composite_system import CompositeSystem
from quara.objects.elemental_system import ElementalSystem
from quara.objects.matrix_basis import get_normalized_pauli_basis
from quara.objects.state import (
    State,
    get_x0_1q,
    get_x1_1q,
    get_y0_1q,
    get_y1_1q,
    get_z0_1q,
    get_z1_1q,
)
from quara.objects.povm import (
    Povm,get_x_measurement,
    get_y_measurement,
    get_z_measurement
)
from quara.objects.gate import (
    Gate, get_depolarizing_channel,
    get_x_rotation,
    get_amplitutde_damping_channel
)
from quara.objects.qoperation import QOperation
from quara.protocol.qtomography.standard.standard_qpt import StandardQpt
from quara.protocol.qtomography.standard.linear_estimator import LinearEstimator
from quara.protocol.qtomography.standard.projected_linear_estimator import (
    ProjectedLinearEstimator,
)

In [2]:
%load_ext autoreload
%autoreload 2

In [4]:
# setup system
e_sys = ElementalSystem(0, get_normalized_pauli_basis())
c_sys = CompositeSystem([e_sys])

# Tester Objects (State)
# |+><+|
state_x0 = get_x0_1q(c_sys)
# |+i><+i|
state_y0 = get_y0_1q(c_sys)
# |0><0|
state_z0 = get_z0_1q(c_sys)
# |1><1|
state_z1 = get_z1_1q(c_sys)
tester_states = [state_x0, state_y0, state_z0, state_z1]

In [5]:
tester_povms = [get_x_measurement(c_sys), get_y_measurement(c_sys), get_z_measurement(c_sys)]

In [8]:
# True Object
true_objects = []
true_objects.append(get_depolarizing_channel(p=0, c_sys=c_sys))
true_objects.append(get_depolarizing_channel(p=0.05, c_sys=c_sys))
true_objects.append(get_depolarizing_channel(p=1, c_sys=c_sys))
true_objects.append(get_x_rotation(theta=np.pi/2, c_sys=c_sys))
true_objects.append(get_x_rotation(theta=np.pi, c_sys=c_sys))
true_objects.append(get_amplitutde_damping_channel(gamma=0.1, c_sys=c_sys))

In [16]:
true_object_names = ["depolarizing_channel_p=0",
"depolarizing_channel_p=0.05",
"depolarizing_channel_p=1",
"x_rotation_theta=np.pi_half",
"x_rotation_theta=np.pi",
"amplitutde_damping_channel_gamma=0.1"]

In [9]:
num_data = [100, 1000]
n_rep = 100

case_name_list = [
    "LinearEstimator(True)",
    "LinearEstimator(False)",
    "ProjectedLinearEstimator(True)",
    "ProjectedLinearEstimator(False)",
]

seed = 777
qtomography_list = [
    StandardQpt(tester_states, tester_povms, on_para_eq_constraint=True, seed=seed),
    StandardQpt(tester_states, tester_povms, on_para_eq_constraint=False, seed=seed),
    StandardQpt(tester_states, tester_povms, on_para_eq_constraint=True, seed=seed),
    StandardQpt(tester_states, tester_povms, on_para_eq_constraint=False, seed=seed),
]
para_list = [True, False, True, False]

estimator_list = [
    LinearEstimator(),
    LinearEstimator(),
    ProjectedLinearEstimator(),
    ProjectedLinearEstimator(),
]

all_results = []

output_dir = f"output_qpt_nrep={n_rep}"
Path(output_dir).mkdir(exist_ok=True)
Path(f"{output_dir}/pickle").mkdir(exist_ok=True)
Path(f"{output_dir}/pdf").mkdir(exist_ok=True)
Path(f"{output_dir}/mse").mkdir(exist_ok=True)

for true_idx, true_object in enumerate(true_objects):
    print("====================================")
    print(f"{true_idx}: {true_object_names[true_idx]}, {true_object}")
    print("====================================")
    estimation_results_list = []
    elapsed_times = []

    for case_idx, name in enumerate(case_name_list):
        qtomography = qtomography_list[case_idx]
        estimator = estimator_list[case_idx]

        start = time.time()
        print(f"Case {case_idx}: {name}")
        print(f"Parametorization: {para_list[case_idx]}")
        print(f"Type of qtomography: {qtomography.__class__.__name__}")
        print(f"Estimator: {estimator.__class__.__name__}")

        print(f"true_object={type(true_object)}")
        estimation_results = data_analysis.estimate(
            qtomography=qtomography,
            true_object=true_object,
            num_data=num_data,
            estimator=estimator,
            iteration=n_rep,
        )
        estimation_results_list.append(estimation_results)

        elapsed_time = time.time() - start
        print("elapsed_time:{0}".format(elapsed_time / 60) + "[min]\n")
        elapsed_times.append(elapsed_time)

    result = dict(
        estimation_results_list=estimation_results_list,
        true_object=true_object,
        elapsed_times=elapsed_times,
    )
    all_results.append(result)
    # output
    with open(
        f"{output_dir}/pickle/qpt_estimation_results_nrep={n_rep}_{true_object_names[true_idx]}.pickle",
        "wb",
    ) as f:
        pickle.dump(result, f)
    # PDF report
    path = (
        f"{output_dir}/pdf/sample_qpt_case{true_idx}_{true_object_names[true_idx]}.pdf"
    )
    report.export_report(
        path,
        result["estimation_results_list"],
        case_name_list,
        estimator_list,
        result["true_object"],
        tester_states + tester_povms,
        seed=seed,
        computation_time=sum(result["elapsed_times"]),
        show_physicality_violation_check=False,
    )
    # Fig
    fig = data_analysis.make_mses_graph_estimation_results(
        result["estimation_results_list"], case_name_list, true_objects[true_idx]
    )
    path = (
        f"{output_dir}/mse/sample_qpt_case{true_idx}_{true_object_names[true_idx]}.html"
    )
    fig.write_html(path)

  1%|          | 1/100 [00:00<00:11,  8.71it/s]

0: Type:	Gate
Dim:	2
HS:	[[1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]]
Case 0: LinearEstimator(True)
Parametorization: True
Type of qtomography: StandardQpt
Estimator: LinearEstimator
true_object=<class 'quara.objects.gate.Gate'>


  2%|▏         | 2/100 [00:00<00:06, 16.11it/s]

elapsed_time:0.12396466334660848[min]

Case 1: LinearEstimator(False)
Parametorization: False
Type of qtomography: StandardQpt
Estimator: LinearEstimator
true_object=<class 'quara.objects.gate.Gate'>


  0%|          | 0/100 [00:00<?, ?it/s]/Users/tomoko/project/rcast/workspace/quara/quara/objects/gate.py:430: ComplexWarning: Casting complex values to real discards the imaginary part
  hs[alpha, beta] = np.trace(np.kron(b_alpha, b_beta) @ choi)


...

elapsed_time:0.09802161852518718[min]

Case 2: ProjectedLinearEstimator(True)
Parametorization: True
Type of qtomography: StandardQpt
Estimator: ProjectedLinearEstimator
true_object=<class 'quara.objects.gate.Gate'>


  0%|          | 0/100 [00:00<?, ?it/s]

elapsed_time:0.6179066975911458[min]

Case 3: ProjectedLinearEstimator(False)
Parametorization: False
Type of qtomography: StandardQpt
Estimator: ProjectedLinearEstimator
true_object=<class 'quara.objects.gate.Gate'>


100%|██████████| 100/100 [00:38<00:00,  2.61it/s]

elapsed_time:0.6393832008043925[min]



In [28]:
import pickle
output_dir = "output"
Path(output_dir).mkdir(exist_ok=True)
with open(f"{output_dir}/qpt_estimation_results_nrep={n_rep}_all.pickle", "wb") as f:
    pickle.dump(all_results, f)

In [19]:
for i, _ in enumerate(all_results):
    print(f"{i}: {true_object_names[i]}")
    print(true_object.hs)
    fig = data_analysis.make_mses_graph_estimation_results(
    all_results[i]["estimation_results_list"],
    case_name_list,
    true_objects[i]
    )
    fig.show()

0: depolarizing_channel_p=0
[[1.        0.        0.        0.       ]
 [0.        0.9486833 0.        0.       ]
 [0.        0.        0.9486833 0.       ]
 [0.1       0.        0.        0.9      ]]


1: depolarizing_channel_p=0.05
[[1.        0.        0.        0.       ]
 [0.        0.9486833 0.        0.       ]
 [0.        0.        0.9486833 0.       ]
 [0.1       0.        0.        0.9      ]]


2: depolarizing_channel_p=1
[[1.        0.        0.        0.       ]
 [0.        0.9486833 0.        0.       ]
 [0.        0.        0.9486833 0.       ]
 [0.1       0.        0.        0.9      ]]


3: x_rotation_theta=np.pi_half
[[1.        0.        0.        0.       ]
 [0.        0.9486833 0.        0.       ]
 [0.        0.        0.9486833 0.       ]
 [0.1       0.        0.        0.9      ]]


4: x_rotation_theta=np.pi
[[1.        0.        0.        0.       ]
 [0.        0.9486833 0.        0.       ]
 [0.        0.        0.9486833 0.       ]
 [0.1       0.        0.        0.9      ]]


5: amplitutde_damping_channel_gamma=0.1
[[1.        0.        0.        0.       ]
 [0.        0.9486833 0.        0.       ]
 [0.        0.        0.9486833 0.       ]
 [0.1       0.        0.        0.9      ]]


In [27]:
for i, _ in enumerate(all_results):
    print(f"{i}: {true_object_names[i]}")
    report.export_report(
        f"{output_dir}/sample_qpt_case{i}_{true_object_names[i]}.pdf",
        all_results[i]["estimation_results_list"],
        case_name_list,
        estimator_list,
        all_results[i]["true_object"],
        tester_states + tester_povms,
        seed=seed,
        computation_time=sum(all_results[i]["elapesed_time"]),
        show_physicality_violation_check=False
    )

0: depolarizing_channel_p=0
​Generating table of computation time ...
​Generating table of experimental conditions ...
Generating case list ...
​​Generating MSE of empirical distributions blocks ...


100%|██████████| 100/100 [00:00<00:00, 21223.01it/s]


​​Generating consictency test blocks ...


/Users/tomoko/project/rcast/workspace/quara/venv/lib/python3.8/site-packages/numpy/core/_methods.py:216: RuntimeWarning:

Degrees of freedom <= 0 for slice

/Users/tomoko/project/rcast/workspace/quara/venv/lib/python3.8/site-packages/numpy/core/_methods.py:209: RuntimeWarning:

invalid value encountered in double_scalars

/Users/tomoko/project/rcast/workspace/quara/quara/objects/gate.py:430: ComplexWarning:

Casting complex values to real discards the imaginary part



​Generating a graph for MSE ...
​​Generating physicality violation test blocks ...
Converting to PDF report ...


...

100%|██████████| 100/100 [00:00<00:00, 37342.45it/s]

​Deleting temporary files ...
Completed to export pdf. (output/sample_qpt_case4_x_rotation_theta=np.pi.pdf)
5: amplitutde_damping_channel_gamma=0.1
​Generating table of computation time ...
​Generating table of experimental conditions ...
Generating case list ...
​​Generating MSE of empirical distributions blocks ...


​​Generating consictency test blocks ...
​Generating a graph for MSE ...


/Users/tomoko/project/rcast/workspace/quara/venv/lib/python3.8/site-packages/numpy/core/_methods.py:216: RuntimeWarning:

Degrees of freedom <= 0 for slice

/Users/tomoko/project/rcast/workspace/quara/venv/lib/python3.8/site-packages/numpy/core/_methods.py:209: RuntimeWarning:

invalid value encountered in double_scalars

/Users/tomoko/project/rcast/workspace/quara/quara/objects/gate.py:430: ComplexWarning:

Casting complex values to real discards the imaginary part



​​Generating physicality violation test blocks ...
Converting to PDF report ...
​Deleting temporary files ...
Completed to export pdf. (output/sample_qpt_case5_amplitutde_damping_channel_gamma=0.1.pdf)


In [29]:
for i, _ in enumerate(all_results):
    print(f"{i}: {true_object_names[i]}")
    fig = data_analysis.make_mses_graph_estimation_results(
    all_results[i]["estimation_results_list"],
    case_name_list,
    true_objects[i]
    )
    path = f"{output_dir}/sample_qpt_case{i}_{true_object_names[i]}.html"
    fig.write_html(path)

0: depolarizing_channel_p=0
1: depolarizing_channel_p=0.05
2: depolarizing_channel_p=1
3: x_rotation_theta=np.pi_half
4: x_rotation_theta=np.pi
5: amplitutde_damping_channel_gamma=0.1
